Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). Do NOT add any cells to the notebook!

Make sure you fill in any place that says `YOUR CODE HERE` or _YOUR ANSWER HERE_ , as well as your name and group below:

In [1]:
NAME = "Christoph Helmberger"
# Pls indicate your student id as a string, w/o the leading 'h'!
student_id = "11915039"

# Assignment 3 (Individual)

In continuation of Assignment 2, we will now practice how to process datasets.
From what you have seen in the lecture you should practice the following steps:
* basic dataset decription & assessing "tidyness"
* filtering
* sorting
* aggregation
using both "pure" Python and also using Pandas. 
-----

## Step 1a (4 points)

Find a CSV dataset online (same requirements apply as in Assignment 2), but additionally make sure that the dataset has at least  one column with a categorical value and one column with a numerical value. Note, if you worked in a group, you as group members are NOT allowed to use the same dataset, i.e., we expect everybody to find their own CSV file.
Save the file into your data folder and name it __data_notebook-1_DataFile.csv__.

To avoid any unexpected errors when running your submission in our grading environment, we highly recommend a dataset which uses the encoding `utf-8`.

Using the CSV package, write the function `analyzeCSV` to return:
* the number of rows in the dataset.
* how many different __values__ and which __datatype__ appears per column in the CSV file.

That is, the function should return a dictionary of the following structure:
```
 {
  "rows": ..., # the number of rows (integer)
  "columns": ... # a list of pairs [ (v_1,dt_1) , (v_2,dt_2), ... (v_1,dt_1)]
 }  
```
where `(v_i,dt_i)` denotes the number of different values (`v_i`), and the datatype (`dt_i`) detected in column `i`.
If you detect several different datatypes per column your function should return the value `"object"` for `dt_i`.

__Hint__: For the (heuristic) datatype detection per column, you can use/adapt the function `convert()` from notebook `unit3/00_value-transformation.ipynb` from the lecture, i.e., possible returned values apart from the string `"object"` should be the return value of the `type()` function applied to the heuristically converted values in the column.

In [2]:
import pandas as pd
import csv
import os
filePath = "./data/data_notebook-1_DataFile.csv"

In [3]:
from datetime import datetime

heuristics = [lambda value: datetime.strptime(value, "%Y-%m-%d"), lambda value: datetime.strptime(value, "%Y%m%d"),
              int, float]

def convert(value):
    for type in heuristics:
        try:
            return type(value)
        except ValueError:
            continue

    return value

def analyzeCSV(filePath):
    with open(filePath, newline='') as csvfile:
        reader = csv.reader(csvfile)
        data = list(reader)

    columns=[]
    header=data[0]
    
    # Counter starts with one because the header row will not appear in the for loop
    rowcount = 1
    for row in data[1:]:
        rowcount += 1
        d={}
        for i,h in enumerate(header):
            d[h]=row[i]
        columns.append(d)

    columns={}
    header=data[0]
    columns={ h:[] for h in header}

    for city in data[1:]:
        for i,h in enumerate(header): 
            if str(city[i]) != "":
                columns[h].append(city[i])
    
    columnValues = []
    for key, value in columns.items():
        columnValues.append({key : set(value)})
    
    returnColumns = []
    for d in columnValues:
        dataType = []
        
        for key, value in d.items():
            for v in value:
                dataType.append(type(convert(v)))
                if len(set(dataType)) > 1:
                    dataType.clear()
                    dataType.append("object")
                    break
                    
        returnColumns.append((len(value), dataType[0]))           
            
    return {"rows": rowcount, "columns": returnColumns}

In [4]:
from nose.tools import assert_equal
assert_equal(type(analyzeCSV(filePath)), dict)
assert_equal(len(analyzeCSV(filePath)), 2)

Now, (potentially using some more manual inspection), answer the following questions:
* Which types of variables appear in this dateset? For each variable, indicate whether
  *  it is nominal or categorical, 
  *  it is an identifier, a dimension or a measurement.  
* How would you describe an "observation" in this dataset?
* Is the dataset tidy?
* If no, why not?

YOUR ANSWER HERE
* Six variables: Reporting country, Reference period, Item, Item code, Value, Date of extraction (yyyymmdd)
* Types of variables:
    * Reporting country: categorical, dimension
    * Reference period: nominal, dimension
    * Item: categorical, dimension
    * Item code: categorical, identifier
    * Value: nominal, measurement
    * Date of extraction (yyyymmdd): nominal, derived variable
    
* Observation: One observation describes the expenses from an insurance company in a specific country as purpose of prudential reporting data

* The dataset is tidy, because there are no values missing, each variable is a column and each observation forms a row

* Dataset:
    * Downloaded on this page under the header "Premiums, claims and expenses": https://data.europa.eu/data/datasets/eiopa-insurance-statistics-solo-annual?locale=en
    * CSV-Link: https://register.eiopa.europa.eu/Publications/Insurance%20Statistics/SA_Own_Funds.csv

## Step 1b (2 points)

Now, perform the same steps you solved in 1a using pandas, i.e., read in the CSV file and write the function `pandalyzeCSV` that computes the same structure as defined in 1a.

__Hint:__ pandas dataframes provide the useful function `nunique()` to compute the number of unique values in a column (`v_i`) as well as the attribute  `dtypes` to get the column datatypes (`dt_i`), which you should use.


In [5]:
def pandalyzeCSV(filePath):
    df_csv = pd.read_csv(filePath)
    
    dictOccurens = {}
    for key, value in df_csv.nunique().items():
        dictOccurens[key] = value
        
    dictType = {}
    for key, value in df_csv.dtypes.items():
        dictType[key] = str(value)

    returnColumns = []
    for key, value in dictOccurens.items():
        returnColumns.append((value, dictType[key]))

    return {"rows": len(df_csv.index), "columns": returnColumns}

In [6]:
from nose.tools import assert_equal
assert_equal(type(pandalyzeCSV(filePath)), dict)
assert_equal(len(pandalyzeCSV(filePath)), 2)

## Step 2a Filtering (2 points)

Using pure Python, i.e. NOT using pandas, write the function `csvFilter` that should:
* convert the CSV to a list of lists,
* filter the rows of the dataset on the numeric column with the most unique values (if there are several columns with the same number of unique values, take the left-most of those) by showing all rows where the value of this column is smaller than the column's mean
* return the list of lists containing only the filtered rows and not containing a header!

In [7]:
def csvFilter(filePath):
    with open(filePath, newline='') as csvfile:
        reader = csv.reader(csvfile)
        data = list(reader)
    
    #get rid of header row
    data.pop(0)
    
    dataInfos = analyzeCSV(filePath)
    
    numColumnIndex = {}
    for d in dataInfos["columns"]:
        if d[1] == int or d[1] == float:          
            if bool(numColumnIndex) == False or d[0] > numColumnIndex["number"]:
                numColumnIndex["number"] = d[0]
                numColumnIndex["index"] = dataInfos["columns"].index(d)
    
    returnList = []
    
    if bool(numColumnIndex) == True:       
        summation = 0.0
        for d in data:
            summation = summation + float(d[numColumnIndex["index"]])
        mean = summation / len(data)
        
        for d in data:
            if float(d[numColumnIndex["index"]]) < mean:
                returnList.append(d)
    
    return returnList

In [8]:
from nose.tools import assert_equal
assert_equal(type(csvFilter(filePath)), list)

## Step 2b Filtering - pandas (2 points)

Now, again using pandas, write the function `pandasFilter` to:
* convert the dataset to a pandas dataframe
* filter the rows of the dataset by the same condition as for 2a:
* return the pandas dataframe only containing the filtered rows!

In [9]:
def pandasFilter(filePath):
    df = pd.read_csv(filePath)
    
    dictOccurens = {}
    for key, value in df.nunique().items():
        dictOccurens[key] = value
        
    dictType = {}
    for key, value in df.dtypes.items():
        dictType[key] = str(value)
    
    dfTypes = df.select_dtypes(include=['int64','float64'])
    
    filterColumn = {}
    
    for c in dfTypes.columns.values:
        if bool(filterColumn) == False or dictOccurens[c] > filterColumn["uniqueValues"]:
            filterColumn["columnName"] = c
            filterColumn["uniqueValues"] = dictOccurens[c]
    
    dfFiltered = df[df[filterColumn["columnName"]] < df[filterColumn["columnName"]].mean()]
    
    return dfFiltered

In [10]:
from nose.tools import assert_equal
assert_equal(type(pandasFilter(filePath)), pd.DataFrame)

## Step 3a Sorting (2 points)

Using pure Python, i.e. NOT using pandas:
* define a function sortCSV(filePath) which sorts the rows of your csv file by the left-most non-numerical in descending order of values. 
* The function takes *filePath* as input (which you have already defined above).
* Your function should return the sorted contents of the CSV file as a list of lists.


In [11]:
def sortCSV(filePath):
    with open(filePath, newline='') as csvfile:
        reader = csv.reader(csvfile)
        data = list(reader)
    
    #get rid of header row
    data.pop(0)
    
    dataInfos = analyzeCSV(filePath)
    
    columnIndex = None
    for d in dataInfos["columns"]:
        if d[1] != int and d[1] != float: 
            columnIndex = dataInfos["columns"].index(d)
            break
    
    if columnIndex != None:
        data.sort(key=lambda x: x[columnIndex])
        data.reverse()
    
    return data

In [12]:
from nose.tools import assert_equal
assert_equal(type(sortCSV(filePath)), list)

## Step 3b Sorting - pandas (2 points)

Now, again using pandas, 
* define a similar function PandasSortCSV(filePath) which sorts the rows of the dataset by the same columns as in 3a, again in descending order of values. 
* This time, your function should return a pandas dataframe.


In [13]:
def PandasSortCSV(filePath):
    df = pd.read_csv(filePath)
    dfTypes = df.select_dtypes(exclude=['int64','float64'])
    dfSorted = df.sort_values(by=[dfTypes.columns.values[0]], ascending=False)
    
    return dfSorted

In [14]:
from nose.tools import assert_equal
assert_equal(isinstance(PandasSortCSV(filePath), pd.DataFrame), True)

## Step 4a Aggregation and Grouping (4 points)

Using pure Python, i.e. NOT using pandas define a function aggregateCSV(filePath) which does the following:
* picks the left-most numeric column _X_ and the left-most non-numeric column _Y_ and
* computes the __sum__ of the values of column _X_ __per__ value of column _Y_
* Your function should return a dictionary holding the unique values of the non-numeric column as keys and the sums of the numeric column as values.

In [15]:
def aggregateCSV(filePath):
    with open(filePath, newline='') as csvfile:
        reader = csv.reader(csvfile)
        data = list(reader)
    
    #get rid of header row
    data.pop(0)
    
    dataInfos = analyzeCSV(filePath)
    
    columnIndexNonNumeric = None
    columnIndexNumeric = None
    for d in dataInfos["columns"]:
        if columnIndexNonNumeric == None and d[1] != int and d[1] != float: 
            columnIndexNonNumeric = dataInfos["columns"].index(d)

        elif columnIndexNumeric == None and (d[1] == int or d[1] == float):
            columnIndexNumeric = dataInfos["columns"].index(d)

        elif columnIndexNonNumeric != None and columnIndexNumeric != None:
            break
    
    groupDict = {}
    if columnIndexNonNumeric != None and columnIndexNumeric != None:
        for d in data:
            if d[columnIndexNonNumeric] in groupDict:
                groupDict[d[columnIndexNonNumeric]] += float(d[columnIndexNumeric])
            else:
                groupDict[d[columnIndexNonNumeric]] = float(d[columnIndexNumeric])
    
    return groupDict

In [16]:
from nose.tools import assert_equal
assert_equal(type(aggregateCSV(filePath)), dict)

## Step 4b Aggregation and Grouping - Pandas (2 points)

Now, again using pandas, do the same as in 4a and call the function `PandasAggregateCSV` this time. It should return a pandas dataframe.


In [17]:
def PandasAggregateCSV(filePath):
    df = pd.read_csv(filePath)
    dfNumeric = df.select_dtypes(include=['int64','float64'])
    dfNonNumeric = df.select_dtypes(exclude=['int64','float64'])
    dfSum = pd.DataFrame(df.groupby(dfNonNumeric.columns.values[0])[dfNumeric.columns.values[0]].sum())
    
    return dfSum

In [18]:
from nose.tools import assert_equal
assert_equal(isinstance(PandasAggregateCSV(filePath), pd.DataFrame), True)